In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import json
import requests
import datetime
from typing import Dict, List, Union
from collections import defaultdict


# Class GameDealFetcher để lấy dữ liệu từ API
class GameDealFetcher:
    def __init__(
        self,
        api_key: str,
        game_id: str,
        shops_file: str,
        country: str = "US",
        shops: str = "61,35,16,6,20,24,37",
        since: str = "2023-11-06T00:00:00Z",
    ):
        """
        Khởi tạo đối tượng GameDealFetcher với các tham số cấu hình để lấy dữ liệu từ API.

        :param api_key: Khóa API để truy cập vào dịch vụ.
        :param game_id: ID của trò chơi cần lấy dữ liệu.
        :param shops_file: Đường dẫn tới tệp JSON chứa thông tin các cửa hàng.
        :param country: Quốc gia mà người dùng muốn lấy dữ liệu, mặc định là "US".
        :param shops: Danh sách các cửa hàng mà người dùng muốn lấy thông tin, mặc định là "61,35,16,6,20,24,37".
        :param since: Thời gian bắt đầu để lấy dữ liệu, mặc định là "2023-11-06T00:00:00Z".
        """
        self.api_key = api_key
        self.base_url = "https://api.isthereanydeal.com/games/history/v2"
        self.params = {
            "key": self.api_key,
            "id": game_id,
            "country": country,
            "shops": shops,
            "since": since,
        }
        self.shops_data = self.load_shops_data(shops_file)

    def load_shops_data(self, shops_file: str) -> Dict:
        """
        Tải dữ liệu cửa hàng từ tệp JSON.

        :param shops_file: Đường dẫn tới tệp JSON chứa thông tin các cửa hàng.
        :return: Dữ liệu cửa hàng dưới dạng từ điển.
        """
        try:
            with open(shops_file, "r", encoding="utf-8") as file:
                return json.load(file)
        except Exception as e:
            raise Exception(f"Lỗi khi đọc tệp JSON cửa hàng: {str(e)}")

    def fetch_deal_history(self) -> Union[List, Dict]:
        """
        Lấy lịch sử giảm giá từ API.

        :return: Dữ liệu lịch sử giảm giá (dạng danh sách hoặc từ điển).
        """
        response = requests.get(self.base_url, params=self.params)
        if response.status_code == 200:
            return response.json()
        else:
            raise Exception(f"Lỗi: {response.status_code} - {response.text}")

    def format_data(self, data: Union[List, Dict]) -> List[Dict]:
        """
        Định dạng dữ liệu từ API thành cấu trúc có tổ chức hơn để dễ xử lý.

        :param data: Dữ liệu từ API trả về (dạng danh sách hoặc từ điển).
        :return: Dữ liệu đã được định dạng dưới dạng danh sách các từ điển.
        """
        if isinstance(data, dict) and "data" in data:
            data = data["data"]

        formatted_data = []

        for item in data:
            shop_id = str(item["shop"]["id"])
            shop_name = self.shops_data.get(shop_id, "Cửa hàng không xác định")
            timestamp = item["timestamp"]
            deal_info = {
                "timestamp": timestamp,
                "price": item["deal"]["price"]["amount"],
                "regular_price": item["deal"]["regular"]["amount"],
                "cut": item["deal"]["cut"],
            }

            shop_entry = next(
                (s for s in formatted_data if s["shop_id"] == shop_id), None
            )

            if not shop_entry:
                shop_entry = {"shop_id": shop_id, "shop_title": shop_name, "deals": []}
                formatted_data.append(shop_entry)

            shop_entry["deals"].append(deal_info)

        return formatted_data

In [2]:
def main():
    # Necessary Information
    api_key = "07b0e806aacf15f38b230a850b424b2542dd71af"
    game_id = "018d937f-590c-728b-ac35-38bcff85f086"
    shops_file = "DiscountFrequencyAnalysis/shops.json"

    # Khởi tạo đối tượng GameDealFetcher
    fetcher = GameDealFetcher(api_key, game_id, shops_file)

    # Lấy dữ liệu lịch sử giảm giá từ API
    data = fetcher.fetch_deal_history()

    # Định dạng dữ liệu thành danh sách các từ điển
    formatted_data = fetcher.format_data(data)

    # Chuyển dữ liệu thành DataFrame
    all_deals = []
    for shop in formatted_data:
        for deal in shop["deals"]:
            all_deals.append({
                "timestamp": deal["timestamp"],  # Giữ nguyên giá trị timestamp
                "shop_id": shop["shop_id"],
                "shop_title": shop["shop_title"],
                "price": deal["price"],
                "regular_price": deal["regular_price"],
                "cut": deal["cut"]
            })

    # Tạo DataFrame từ danh sách các giao dịch
    df = pd.DataFrame(all_deals)

    # Hiển thị DataFrame
    print(df)

    return df


if __name__ == "__main__":
    df = main()

                     timestamp shop_id        shop_title  price  \
0    2024-11-24T19:25:51+01:00       6             Steam  52.68   
1    2024-11-23T17:06:10+01:00       6             Steam  59.99   
2    2024-09-28T10:25:07+02:00       6             Steam  52.68   
3    2024-09-27T10:25:59+02:00       6             Steam  53.99   
4    2024-09-26T19:05:23+02:00       6             Steam  59.99   
..                         ...     ...               ...    ...   
141  2024-02-11T00:55:13+01:00      24  Epic Games Store  59.99   
142  2024-01-04T19:35:08+01:00      24  Epic Games Store  59.99   
143  2023-12-22T12:35:06+01:00      24  Epic Games Store  35.99   
144  2023-12-21T20:40:05+01:00      24  Epic Games Store  31.67   
145  2023-11-06T01:00:00+01:00      24  Epic Games Store  59.99   

     regular_price  cut  
0            59.99   12  
1            59.99    0  
2            59.99   12  
3            59.99   10  
4            59.99    0  
..             ...  ...  
141          